In [12]:
# Import libraries
# t-test libraries at the end 

import requests
import pandas as pd
import json
import os
import arcpy
import numpy as np
from scipy import stats 
from numpy.random import seed 
from numpy.random import randn 
from numpy.random import normal 
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt


In [5]:
# define the workspace so the files write to this spot
workspace = r'Z:\Helpdesk\Shana\GIS 5572\5572Final\5572Final.gdb'

In [3]:
# Define field names to pull from the census data into the dataframe
field_names = ['GEOID', 'State', 'County', 'P0010001', 'P005_calc_pct0004', 'mean_delay_saturday', 'mean_saturday_status', 'Point_Count']

In [6]:
# Create a numpy array in order to create a dataframe
array1 = arcpy.da.FeatureClassToNumPyArray('BlockGroupselection_SummarizeWithin', field_names)

In [7]:
# create a dataframe from the array
df = pd.DataFrame(array1)

In [8]:
# look at the first rows. I see North Dakota! Yikes!
df.head()

,GEOID,State,County,P0010001,P005_calc_pct0004,mean_delay_saturday,mean_saturday_status,Point_Count
0,380719576001,North Dakota,Ramsey County,1507,0.5,NaN,NaN,0
1,380719576002,North Dakota,Ramsey County,793,0.9,NaN,NaN,0
2,380719576003,North Dakota,Ramsey County,887,1.2,NaN,NaN,0
3,380719576004,North Dakota,Ramsey County,1206,0.1,NaN,NaN,0
4,380719577001,North Dakota,Ramsey County,986,1.5,NaN,NaN,0


In [9]:
# Select only the Minnesota data
df = df[df['State'] == 'Minnesota']

In [ ]:
# double check
df.head()

In [ ]:
# Select only the block groups that have stops in them
df = df[df['Point_Count'] != 0]

In [ ]:
# Create a new dataframe  - I was thinking about pulling in the geometry, but that doesn't work!
# I can't remember why I wanted to make this dataframe.... The data I want is in the Summarize Within layer
selection = arcpy.da.FeatureClassToNumPyArray('BlockGroupselection_Features', ('GEOID', 'State', 'County', 'P0010001', 'P005_calc_pct0004', 'Shape_Length', 'Shape_Area') )

In [ ]:
original_df = pd.DataFrame(selection)

In [ ]:
original_df.head()

In [ ]:
original_df = original_df[original_df['State'] == 'Minnesota']

## Student TTest
In this anaylsis, my null hypothesis is that delays in geographies (block groups or tracts) with high percentage of Black population is the same as in block groups or tracts with lower percentanges of Black population.

alternative hypothesis: dealys in the geographies (block groups or tracts) with high percentage of Black population is HIGHER or LOWER than in block groups or tracts with lower percentanges of Black population.


Reference: https://thedatascientist.com/how-to-do-a-t-test-in-python/

In [10]:
columns = df.columns.tolist()
print(columns)

['GEOID', 'State', 'County', 'P0010001', 'P005_calc_pct0004', 'mean_delay_saturday', 'mean_saturday_status', 'Point_Count']


In [11]:
# Calculate the mean of the dealy in the block groups. Just for fun.
mean_value = df['P005_calc_pct0004'].mean()
std_deviation = df['P005_calc_pct0004'].std()

print("Mean:", mean_value)
print("Standard Deviation:", std_deviation)

Mean: 12.669036845507433
Standard Deviation: 14.195278875199875


In [13]:
# create a histogram of the delay. Why? Just because it's interesting. To explore the data.
plt.hist(array1['P005_calc_pct0004'], bins=10)  # Adjust the number of bins as needed
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Column "{}"'.format('column_name'))  # Replace 'column_name' with the actual column name
plt.show()

In [14]:
# create a new df with only the block groups with the highest percentage of African American population (the top 25% )
percent_pop = df['P005_calc_pct0004']

# Calculate the 75th percentile value
threshold = pd.Series(percent_pop).quantile(0.75)

# Select rows where the field value is greater than or equal to the threshold
top_25_percent = df[percent_pop >= threshold]

In [15]:
top_25_percent.head()

,GEOID,State,County,P0010001,P005_calc_pct0004,mean_delay_saturday,mean_saturday_status,Point_Count
12,270530001011,Minnesota,Hennepin County,1057,34.0,1.500000,0.857143,14
13,270530001012,Minnesota,Hennepin County,812,27.1,NaN,NaN,0
14,270530001013,Minnesota,Hennepin County,777,27.2,3.769231,1.000000,13
15,270530001014,Minnesota,Hennepin County,705,25.5,NaN,NaN,0
16,270530001021,Minnesota,Hennepin County,1128,37.2,3.000000,0.956522,46


In [16]:
# count the number of rows in the subset
num_rows = len(top_25_percent)  # Using len() function
print("Number of rows:", num_rows)

Number of rows: 387


In [17]:
# count the number of rows in the total population
num_rows = len(df)  # Using len() function
print("Number of rows:", num_rows)

Number of rows: 1569


In [18]:
# Finding the mean of the sample selection
mean_value_sample = top_25_percent['P005_calc_pct0004'].mean()
std_deviation_sample = top_25_percent['P005_calc_pct0004'].std()

print("Mean:", mean_value_sample)
print("Standard Deviation:", std_deviation_sample)

Mean: 32.86434108527132
Standard Deviation: 14.213558795917983


In [24]:
# Select only the block groups that have stops in them. Eliminate all block groups that have 0 in the Point Count field
df = df[df['Point_Count'] != 0]
top_25_percent = top_25_percent[top_25_percent['Point_Count'] != 0]

In [25]:
# Create lists from the "mean_delay_saturday" field to use in the ttest
sample_list = top_25_percent['mean_delay_saturday'].tolist()
whole_list = df['mean_delay_saturday'].tolist()

In [26]:
# Calculate the ttest
t_statistic, p_value = ttest_ind(sample_list, whole_list)

print("t-statistic:", t_statistic)
print("p-value:", p_value)

t-statistic: 0.2851648550600439
p-value: 0.7755611690293546


## Results
With the results of:
* t-statistic: 0.2851648550600439
* p-value: 0.7755611690293546

We can conclude that the null hypothesis is correct: there is no statistical difference in the mean delay time between Block Groups with high African American population and the set of block groups as a whole.. 